In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType,DecimalType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
spark = (SparkSession
 .builder
 .appName("Fact")
 .getOrCreate())

22/08/27 11:13:47 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/08/27 11:13:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/27 11:13:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/27 11:14:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/27 11:14:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/27 11:14:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/27 11:14:05 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [7]:
profession_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Beautiful_Soap/Profession/*.csv')

state_city_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Beautiful_Soap/State/*.csv')

state_salary_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Beautiful_Soap/State_Salary_Deviation/*.csv')



In [8]:
profession_df.show()

+--------------------+------+---+
|          Profession|Salary| Id|
+--------------------+------+---+
|              Design| 48574|  1|
|           Logistics| 50358|  2|
|            Research| 56399|  3|
|        Construction| 57270|  4|
|                  HR| 58029|  5|
|Distribution and ...| 59591|  6|
|          Production| 60046|  7|
|       Marketing, PR| 60174|  8|
|                  IT| 60563|  9|
|            Engineer| 62564| 10|
|             Banking| 62744| 11|
|     Business Advice| 64173| 12|
|              Lawyer| 68642| 13|
|             Finance| 73847| 14|
|              Doctor| 89539| 15|
+--------------------+------+---+



In [9]:
state_city_df.show()

+--------------------+-----------------+---+
|                City|            State| Id|
+--------------------+-----------------+---+
|          Reutlingen|Baden-Württemberg|  1|
|                 Ulm|Baden-Württemberg|  2|
|           Pforzheim|Baden-Württemberg|  3|
|           Stuttgart|Baden-Württemberg|  4|
|           Karlsruhe|Baden-Württemberg|  5|
|Freiburg im Breisgau|Baden-Württemberg|  6|
|           Heilbronn|Baden-Württemberg|  7|
|          Heidelberg|Baden-Württemberg|  8|
|            Mannheim|Baden-Württemberg|  9|
|          Regensburg|          Bavaria| 10|
|    Munich (München)|          Bavaria| 11|
|Nuremberg (Nürnberg)|          Bavaria| 12|
|            Würzburg|          Bavaria| 13|
|          Ingolstadt|          Bavaria| 14|
|               Fürth|          Bavaria| 15|
|            Augsburg|          Bavaria| 16|
|            Erlangen|          Bavaria| 17|
|              Berlin|           Berlin| 18|
|             Potsdam|      Brandenburg| 19|
|         

In [10]:
state_salary_df.show()

+--------------------+----------------+---+
|               State|Salary_Deviation| Id|
+--------------------+----------------+---+
|   Baden-Württemberg|             7.9|  1|
|             Bavaria|             4.4|  2|
|              Berlin|            -5.7|  3|
|         Brandenburg|           -18.8|  4|
|              Bremen|            -2.9|  5|
|             Hamburg|             5.5|  6|
|               Hesse|             7.2|  7|
|        Lower Saxony|            -6.8|  8|
|Mecklenburg-Vorpo...|           -22.0|  9|
|North Rhine-Westp...|             1.1| 10|
|Rhineland-Palatinate|            -1.4| 11|
|            Saarland|            -4.4| 12|
|              Saxony|           -18.5| 13|
|       Saxony-Anhalt|           -19.5| 14|
|  Schleswig-Holstein|            -9.8| 15|
|           Thuringia|           -17.0| 16|
+--------------------+----------------+---+



In [30]:
state_salary_df.createOrReplaceTempView('state_salary_table')
state_city_df.createOrReplaceTempView('state_city_table')
profession_df.createOrReplaceTempView('profession_table')

state_city_salary = spark.sql("SELECT t1.State,t1.Salary_Deviation,t2.City FROM state_salary_table t1 INNER JOIN state_city_table t2 ON t1.State LIKE t2.State")

#fact = state_salary_df.select('State','State_Salary_Deviation','City').join(state_city_df, state_salary_df.State == state_city_df.State).show()

In [28]:
state_city_salary.createOrReplaceTempView('state_city_salary_table')

In [51]:


final_df = spark.sql("SELECT t1.Profession, t1.Salary, t2.*,(Salary + (Salary*Salary_Deviation)/100 ) as Sal_Dev FROM profession_table as t1 CROSS JOIN state_city_salary_table as t2")

In [52]:
final_df.write.format('csv').option('header','true').mode('overwrite').option('encoding','UTF-8').save('Fact')